In [76]:
import base64
import email
import os
import re

from lxml import etree

In [82]:
directory = 'emails'
for filename in os.listdir(directory):
    if not filename.endswith('.eml'): 
        continue
    path = os.path.join(directory, filename)
    
    with open(path, 'r') as eml_file:
        msg = email.message_from_file(eml_file)

    if not 'duolingo' in msg['From'].casefold():
        continue

    print(msg['date'])
    for part in msg.walk():
        if part.get_content_type() != 'text/html':
            continue
        if part.get('Content-Transfer-Encoding') != 'base64':
            continue
            
        # old format
        content = base64.b64decode(part.get_payload()).decode()
        tree = etree.HTML(content)
        results = tree.xpath(
            "//*[contains(text(),'Lessons finished')]/ancestor::tr[1]/following-sibling::tr[1]//b[@class='card-value chameleon-heading']/text()"
        )
        if len(results) == 1:
            print(int(results[0].strip()))
            continue

        # new format
        match = re.search(r'(\d+)\s*lessons?', content.strip(), re.IGNORECASE)
        if match:  
            print(int(match.group(1)))
            continue

Sat, 28 Dec 2024 17:27:55 +0000
154
Sat, 2 Sep 2023 04:14:45 +0000
14
